# Inception上的测试

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import numpy as np
from sklearn.utils import shuffle
import pandas as pd

import h5py
import os
import shutil

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


## 数据处理
读取训练和测试图片，格式化成（299,299,3）的格式。并且划分训练集和验证集

In [2]:
import numpy as np
from tqdm import tqdm
import cv2

np.random.seed(2017)

n = 24962
X = np.zeros((n, 299, 299, 3), dtype=np.uint8)
y = np.zeros((n, 1), dtype=np.uint8)

train_cat2 = os.listdir("train2/cat")
train_dog2 = os.listdir("train2/dog")

i = 0
for filename in train_cat2:
    X[i] = cv2.resize(cv2.imread('train/%s' % filename), (299, 299))
    y[i] = 0
    i += 1
for filename in train_dog2:
    X[i] = cv2.resize(cv2.imread('train/%s' % filename), (299, 299))
    y[i] = 1
    i += 1

In [3]:
m = 12500
X_test = np.zeros((m, 299, 299, 3), dtype=np.uint8)

test_file = os.listdir("test")
j = 0
for filename in test_file:
    X_test[j] = cv2.resize(cv2.imread('test/%s' % filename), (299, 299))
    j += 1

In [4]:
print("i=", i)
print("X_test size:", X_test.shape)

i= 24962
X_test size: (12500, 299, 299, 3)


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

## 模型一
去除顶层，锁定其他所有层，不参与训练；在这模型的基础上，加上GAP层和Dropout层，加sigmoid激活函数得出二分类的结果

In [6]:
base_model = InceptionV3(input_tensor=Lambda(inception_v3.preprocess_input)(Input((299, 299, 3))), weights='imagenet', include_top=False)

for layers in base_model.layers:
    layers.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(base_model.input, x)

In [7]:
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_valid, y_valid))

Train on 19969 samples, validate on 4993 samples
Epoch 1/5
19969/19969 [==============================] - 83s 4ms/step - loss: 0.1943 - acc: 0.9282 - val_loss: 0.0355 - val_acc: 0.9920
Epoch 2/5
19969/19969 [==============================] - 78s 4ms/step - loss: 0.1171 - acc: 0.9578 - val_loss: 0.1437 - val_acc: 0.9465
Epoch 3/5
19969/19969 [==============================] - 78s 4ms/step - loss: 0.1132 - acc: 0.9579 - val_loss: 0.0684 - val_acc: 0.9784
Epoch 4/5
19969/19969 [==============================] - 78s 4ms/step - loss: 0.1072 - acc: 0.9603 - val_loss: 0.0985 - val_acc: 0.9666
Epoch 5/5
19969/19969 [==============================] - 78s 4ms/step - loss: 0.1099 - acc: 0.9588 - val_loss: 0.0325 - val_acc: 0.9924


In [9]:
for i in range(len(model.layers)):
    print(i,model.layers[i].name)

0 input_1
1 lambda_1
2 conv2d_1
3 batch_normalization_1
4 activation_1
5 conv2d_2
6 batch_normalization_2
7 activation_2
8 conv2d_3
9 batch_normalization_3
10 activation_3
11 max_pooling2d_1
12 conv2d_4
13 batch_normalization_4
14 activation_4
15 conv2d_5
16 batch_normalization_5
17 activation_5
18 max_pooling2d_2
19 conv2d_9
20 batch_normalization_9
21 activation_9
22 conv2d_7
23 conv2d_10
24 batch_normalization_7
25 batch_normalization_10
26 activation_7
27 activation_10
28 average_pooling2d_1
29 conv2d_6
30 conv2d_8
31 conv2d_11
32 conv2d_12
33 batch_normalization_6
34 batch_normalization_8
35 batch_normalization_11
36 batch_normalization_12
37 activation_6
38 activation_8
39 activation_11
40 activation_12
41 mixed0
42 conv2d_16
43 batch_normalization_16
44 activation_16
45 conv2d_14
46 conv2d_17
47 batch_normalization_14
48 batch_normalization_17
49 activation_14
50 activation_17
51 average_pooling2d_2
52 conv2d_13
53 conv2d_15
54 conv2d_18
55 conv2d_19
56 batch_normalization_13
57

In [10]:
model.save_weights('inceptionv3_weights_1.h5')  
y_pred = model.predict(X_test, verbose=1)

12500/12500 [==============================] - 42s 3ms/step


In [12]:
y_pred = y_pred.clip(min=0.005, max=0.995)
df = pd.read_csv("sample_submission.csv")
for i, fname in enumerate(test_file):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('inceptionv3_predict_1.csv', index=None)
df.head(2)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,id,label
0,1,0.995
1,2,0.995


在kaggle上的得分是0.05805

# 模型二
fine-tuning 锁前161层，放开161层之后的层参与训练

In [13]:
for layers in base_model.layers:
    layers.trainable = False
for layer in model.layers[161:]:
    layer.trainable = True
model.load_weights('inceptionv3_weights_1.h5')
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_valid, y_valid))
model.save_weights('inceptionv3_weights_2.h5') 

Train on 19969 samples, validate on 4993 samples
Epoch 1/5
19969/19969 [==============================] - 120s 6ms/step - loss: 0.0741 - acc: 0.9746 - val_loss: 0.0509 - val_acc: 0.9880
Epoch 2/5
19969/19969 [==============================] - 111s 6ms/step - loss: 0.0263 - acc: 0.9911 - val_loss: 0.0456 - val_acc: 0.9874
Epoch 3/5
19969/19969 [==============================] - 111s 6ms/step - loss: 0.0103 - acc: 0.9962 - val_loss: 0.0487 - val_acc: 0.9872
Epoch 4/5
19969/19969 [==============================] - 111s 6ms/step - loss: 0.0074 - acc: 0.9979 - val_loss: 0.0472 - val_acc: 0.9864
Epoch 5/5
19969/19969 [==============================] - 111s 6ms/step - loss: 0.0032 - acc: 0.9988 - val_loss: 0.0346 - val_acc: 0.9914


In [15]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)
df = pd.read_csv("sample_submission.csv")
for i, fname in enumerate(test_file):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('inceptionv3_predict_2.csv', index=None)
df.head(2)

12500/12500 [==============================] - 43s 3ms/step


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,id,label
0,1,0.995
1,2,0.995


在kaggle上的得分是0.05087

## 模型三
fine-tuning 锁前252层，放开252层之后的层参与训练

In [16]:
for layers in base_model.layers:
    layers.trainable = False
for layer in model.layers[252:]:
    layer.trainable = True
model.load_weights('inceptionv3_weights_2.h5')
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [17]:
model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_valid, y_valid))

Train on 19969 samples, validate on 4993 samples
Epoch 1/5
19969/19969 [==============================] - 94s 5ms/step - loss: 0.0030 - acc: 0.9988 - val_loss: 0.0313 - val_acc: 0.9924
Epoch 2/5
19969/19969 [==============================] - 89s 4ms/step - loss: 0.0014 - acc: 0.9995 - val_loss: 0.0333 - val_acc: 0.9926
Epoch 3/5
19969/19969 [==============================] - 89s 4ms/step - loss: 0.0010 - acc: 0.9996 - val_loss: 0.0335 - val_acc: 0.9922
Epoch 4/5
19969/19969 [==============================] - 89s 4ms/step - loss: 5.1250e-04 - acc: 0.9999 - val_loss: 0.0339 - val_acc: 0.9924
Epoch 5/5
19969/19969 [==============================] - 89s 4ms/step - loss: 8.3346e-04 - acc: 0.9997 - val_loss: 0.0348 - val_acc: 0.9920


In [18]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)
df = pd.read_csv("sample_submission.csv")
for i, fname in enumerate(test_file):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('inceptionv3_predict_3.csv', index=None)
df.head(2)

12500/12500 [==============================] - 43s 3ms/step


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,id,label
0,1,0.995
1,2,0.995


In [19]:
model.save_weights('inceptionv3_weights_3.h5') 

在kaggle上的得分是0.04785